### Discriminating cryo-EM particles in images
👋 Welcome to my cryo-EM project! This Python-based toolkit is designed to facilitate the reconstruction process in cryo-electron microscopy (cryo-EM), and is comprised of three main functions:

* Step 1: Clustering of data (images)
* Step 2: Particle Picking
* Step 3: 2D Classification
